## 加载模型

In [ ]:
!export CUDA_VISIBLE_DEVICES=4,5,6,7
from lmdeploy import pipeline, GenerationConfig, TurbomindEngineConfig
import json


model_path="/chubao/tj-train-ssd-21/wanghaotian/scripts/LLaMA-Factory/saves/qwen2-72B/qwen2_72b_dashu_v5_231w_func/checkpoint-4678"
backend_config = TurbomindEngineConfig(tp=4)
gen_config = GenerationConfig(top_p=0.8,
                              top_k=1,
                              temperature=0.8,
                              max_new_tokens=1024)
pipe = pipeline(model_path,
                backend_config=backend_config)


## sys and promt

In [ ]:

func_list=[
            {"id":"456",
            "type": "function",
            "function":{
                'name': 'generate_invoice', 
                'description': '生成发票', 
                'parameters': {
                    'type': 'object', 
                    'properties': {
                        'customer_name': {
                            'type': 'string', 'description': '客户名称'
                        }, 
                        'items': {
                            'type': 'array', 
                            'items': {
                                'type': 'object', 
                                'properties': {
                                    'name': {
                                        'type': 'string', 'description': 'The item name'
                                    }, 
                                    'quantity': {
                                        'type': 'integer', 'description': 'The quantity of the item'
                                    }, 
                                    'price': {
                                        'type': 'number', 'description': 'The price per unit'
                                    }
                                }, 
                            'required': ['name', 'quantity', 'price']
                            }
                        }
                    }, 
                    'required': ['customer_name', 'items']
                }
            }
            },
            {"id":"123",
            "type": "function",
            "function":{'name': 'generate_password', 'description': '生成随机密码', 'parameters': {'type': 'object', 'properties': {'length': {'type': 'integer', 'description': '密码的长度'}}, 'required': ['length']}}}
]

# func_list=[
#         {   "id":"456",
#             "type": "function",
#             "function": {
#                 "name": "current_time",
#                 "description": "返回当前事件",
#                 "parameters": {}
#             }
#         }
#     ]

system = """You are a helpful assistant.You have access to the following functions:
{functions}
If a you choose to call a function ONLY reply in the following format:
{start_tag}{function_name}{parameters}{end_tag}
where:
start_tag => `<functioncall>`
function_name => name of the function
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</functioncall>`

Here is an example:
<functioncall> eaxmple_function_name {"example_arguments": "example_value"} </functioncall>

Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
"""
sys_input = system.replace("{functions}", "\n".join([json.dumps(x, ensure_ascii=False) for x in func_list]))

prompt="我需要为John Doe生成一张发票。他购买了2个苹果，每个$1，以及3根香蕉，每根$0.5。"
# prompt = "现在是几点?"
print(sys_input)

## 解析函数

In [ ]:
        
from lmdeploy.serve.openai.protocol import (  # noqa: E501
    ChatCompletionRequest, ChatCompletionResponse,
    ChatCompletionResponseChoice, ChatCompletionResponseStreamChoice,
    ChatCompletionStreamResponse, ChatCompletionTokenLogprob, ChatMessage,
    ChoiceLogprobs, CompletionRequest, CompletionResponse,
    CompletionResponse, CompletionResponseChoice,
    CompletionResponseStreamChoice, CompletionStreamResponse, DeltaMessage,
    EmbeddingsRequest, EncodeRequest, EncodeResponse, ErrorResponse,
    FunctionResponse, FunctionStreamResponse,GenerateRequest, GenerateResponse,
    LogProbs, ModelCard, ModelList, ModelPermission, ToolCall, TopLogprob,
    UsageInfo,ToolCallStream,FunctionResponse)  
import dataclasses
from typing import AsyncGenerator, Dict, List, Literal, Optional, Union

@dataclasses.dataclass
class UnmarshalRes:
    response_json: str
    first_return: bool
    last_return: bool
    name: str
    action_id: int 
def create_stream_response_json(
        index: int,
        text: str,
        tool_calls: Optional[List[ToolCall]],
        finish_reason: Optional[str] = None,
        logprobs: Optional[LogProbs] = None) -> str:
    choice_data = ChatCompletionResponseStreamChoice(
        index=index,
        delta=DeltaMessage(role='assistant', content=text,tool_calls=tool_calls),
        finish_reason=finish_reason,
        logprobs=logprobs)
    response = ChatCompletionStreamResponse(
        id="1",
        created="1",
        model="test",
        choices=[choice_data]
    )
    response_json = response.model_dump_json()

    return response_json

def unmarshal_llama3_1_tool(res,tmp_prefix_result,first_return,last_return,name,action_id,logprobs):
        if res.finish_reason == 'stop':
            res.finish_reason = 'tool_calls'
        # print(tmp_prefix_result)
        
        if '<functioncall>' in tmp_prefix_result and ' {' in tmp_prefix_result:   
            if name=="":
                name = tmp_prefix_result.split('<functioncall> ')[1].split(' {')[0]
                action_id = [tool['function']['name'] for tool in func_list].index(name)
        else:
            return None

        if not first_return:
            fisrt_arguments=tmp_prefix_result.split(name+' ')[1]
            tool_calls = [
                ToolCall(index=str(action_id),
                    id=str(action_id),
                    function=FunctionResponse(name=name,arguments=fisrt_arguments))]
            response_json = create_stream_response_json(
                    index=0,
                    text='',
                    tool_calls=tool_calls,
                    finish_reason=res.finish_reason,
                    logprobs=logprobs)
            first_return=True
        elif not last_return:
            if tmp_prefix_result.endswith(' </'):
                tool_response=res.text.split(' </')[0]
                tool_calls = [
                    ToolCallStream(index=str(action_id),function=FunctionStreamResponse(arguments=tool_response))]
                response_json = create_stream_response_json(
                        index=0,
                        text='',
                        tool_calls=tool_calls,
                        finish_reason=res.finish_reason,
                        logprobs=logprobs)
                last_return=True
            else:
                # if tmp_prefix_result.endswith(' {"'):
                #     res.text='{"'+res.text
                tool_calls = [
                    ToolCallStream(index=str(action_id),function=FunctionStreamResponse(arguments=res.text))]
                response_json = create_stream_response_json(
                        index=0,
                        text='',
                        tool_calls=tool_calls,
                        finish_reason=res.finish_reason,
                        logprobs=logprobs)
        else:
            response_json = create_stream_response_json(
                    index=0,
                    text=None,
                    tool_calls=None,
                    finish_reason=res.finish_reason,
                    logprobs=logprobs)
        return UnmarshalRes(response_json,first_return,last_return,name,action_id)


## 访问测试

In [ ]:
import nest_asyncio
nest_asyncio.apply()
messages=[
    {
        "role": "system",
        "content": sys_input
    },
    {
        "role": "user",
        "content": prompt
    }
]

init_unmarshal=UnmarshalRes("",False,False,"",None)
tmp_prefix_result=""
accumulated_result=""
for item in pipe.stream_infer(messages, gen_config=gen_config):
    tmp_prefix_result += item.text
    logprobs, usage = None, None
    model_unmarshal_res=unmarshal_llama3_1_tool(item,tmp_prefix_result,init_unmarshal.first_return,init_unmarshal.last_return,init_unmarshal.name,init_unmarshal.action_id,logprobs)
    if model_unmarshal_res!=None:
        response_json=model_unmarshal_res.response_json
        init_unmarshal=model_unmarshal_res
    else:
        continue
    res_josn=json.loads(model_unmarshal_res.response_json)
    print(res_josn)
    if res_josn['choices'][0]['delta']['tool_calls']!=None:
        if 'id' in res_josn['choices'][0]['delta']['tool_calls'][0]:
            res_tool=res_josn['choices'][0]['delta']['tool_calls'][0]['function']
            accumulated_result=accumulated_result+res_josn['choices'][0]['delta']['tool_calls'][0]['function']['arguments']
        else:
            res_tool=res_josn['choices'][0]['delta']['tool_calls'][0]['function']['arguments']
            accumulated_result=accumulated_result+res_josn['choices'][0]['delta']['tool_calls'][0]['function']['arguments']
print(accumulated_result)